In [1]:
from monai.data import Dataset, DataLoader
import numpy as np
import os

class CustomDataset(Dataset):
    def __init__(self, image_files, label_files, transform=None):
        self.image_files = image_files
        self.label_files = label_files
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image = np.load(self.image_files[index])
        label = np.load(self.label_files[index])

        if self.transform:
            image, label = self.transform(image, label)

        return image, label


In [2]:
from monai.transforms import Compose, Resize, ToTensor

transform = Compose([
    Resize((128, 128, 128)),
    ToTensor()
])


In [3]:
from monai.transforms import Transform

class ConcatRandomTensor(Transform):
    def __init__(self):
        super().__init__()

    def __call__(self, image, label):
        random_tensor = np.random.rand(*image.shape)
        new_image = np.concatenate([image, random_tensor], axis=0)
        return new_image, label


In [7]:
from monai.networks.nets import UNet

net = UNet(
    dimensions=3,
    in_channels=2,      # Change this based on concatenated tensor
    out_channels=3,     # Define the number of classes for segmentation
    channels=(16, 32, 64, 128, 256),  # Number of feature channels in each layer
    strides=(2, 2, 2, 2),  # Strides for down-sampling layers
    num_res_units=2,     # Number of residual units in each layer
    norm='batch',        # Normalization type: 'batch', 'instance', or 'group'
)


monai.networks.nets.unet UNet.__init__:dimensions: Argument `dimensions` has been deprecated since version 0.6. Please use `spatial_dims` instead.


In [1]:

from monai.engines import SupervisedTrainer
from monai.losses import DiceLoss
from torch.optim import Adam


from monai.data import DataLoader

# Assuming you have created the CustomDataset and applied transformations as mentioned before
train_dataset = CustomDataset(image_files, label_files, transform)

batch_size = 2  # Adjust as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


device = "cuda" if torch.cuda.is_available() else "cpu"
loss_function = DiceLoss(to_onehot_y=True, softmax=True)
optimizer = Adam(net.parameters(), lr=1e-4)

trainer = SupervisedTrainer(
    device=device,
    max_epochs=100,
    train_data_loader=train_loader,
    network=net,
    optimizer=optimizer,
    loss_function=loss_function,
    inferer=None,
    amp=True,  # Automatic Mixed Precision
)

trainer.run()


NameError: name 'CustomDataset' is not defined